In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns


In [2]:
#importing the datasets
sourcewisedata=pd.read_csv("Daily_Power_Gen_Source_march_23.csv")
statewisedata=pd.read_csv("Daily_Power_Gen_States_march_23.csv")
sourcewisedata.head(10)

,source,NR,WR,SR,ER,NER,All India,date
0,Hydro,139.0,43.0,72.0,30.0,7.0,292,2013-03-31
1,Total,675.0,820.0,697.0,306.0,28.0,2526,2013-03-31
2,Wind Gen(MU),2.0,19.0,13.0,0.0,0.0,34,2013-03-31
3,Hydro,137.0,43.0,83.0,32.0,5.0,300,2013-04-01
4,Total,683.0,841.0,706.0,316.0,29.0,2575,2013-04-01
5,Wind Gen(MU),7.0,21.0,19.0,0.0,0.0,48,2013-04-01
6,Wind Gen(MU),8.0,25.0,15.0,0.0,0.0,48,2013-04-02
7,Total,675.0,836.0,709.0,311.0,28.0,2558,2013-04-02
8,Hydro,136.0,42.0,79.0,29.0,4.0,290,2013-04-02
9,Total,676.0,835.0,705.0,307.0,28.0,2551,2013-04-03


In [ ]:
#check the shape of both the datasets
print(sourcewisedata.shape)

In [ ]:
sourcewisedata.info()

In [ ]:
#see the categories within the source wise dataset
sourcewisedata["source"]= sourcewisedata["source"].astype("category")
sourcewisedata["source"].cat.categories

In [ ]:
#removing the Total type of category rows, because it is the total accross the column
sourcewisedata=sourcewisedata[sourcewisedata['source']!='Total']
sourcewisedata.head(10)

In [ ]:
sourcewisedata["source"]=sourcewisedata['source'].cat.remove_categories(['Total'])

In [ ]:
#Rename the categories so that we can map each of them with carbon foot print values
sourcewisedata["source"]=sourcewisedata["source"].replace({'Coal':"Coal & Lignite", 'Gas, Naptha & Diesel':"Gas", 'Lignite':"Coal & Lignite",
       'RES (Wind, Solar, Biomass & Others)':"RES", 'Solar Gen (MU)':"Solar",
       'Thermal (Coal & Lignite)':"Coal & Lignite", 'Wind Gen(MU)':"Wind"})
sourcewisedata["source"].cat.categories


In [ ]:
#Visualizing the trends in energy generation and carbon emission across different regions in India
sourcewisedata['date']=pd.to_datetime(sourcewisedata['date'])
sourcewisedata['Year']=sourcewisedata['date'].dt.year
sourcewisedata.head()

In [ ]:
#according to World Nuclear Association the amount of carbon footprint (in grams) released through differenct sources of energy in producing a KWh of electricity is as follows:
CO2_emissions_mapping={"Hydro":24,"Wind":12,"Nuclear":12,"Coal & Lignite":820,"Solar":44,"Gas":490,"RES":64} #note that the RES key has the value of average of all the other renewable enrgy sources (Hydro, Nuclear, Solar etc) including Biomass
#insert a column corresponding to each source type to get the corresponding carbon emission for that source type with the help of our carbon_emissions_mapping dictionary
sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"]=sourcewisedata["source"].map(CO2_emissions_mapping)
sourcewisedata["NR_CO2 (Million Tonnes)"]=(sourcewisedata["NR"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata["WR_CO2 (Million Tonnes)"]=(sourcewisedata["WR"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata["SR_CO2 (Million Tonnes)"]=(sourcewisedata["SR"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata["ER_CO2 (Million Tonnes)"]=(sourcewisedata["ER"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata["NER_CO2 (Million Tonnes)"]=(sourcewisedata["NER"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata["All_India_CO2 (Million Tonnes)"]=(sourcewisedata["All India"]* sourcewisedata["CO2_emissions_due_to_source_type(gCO2e/KWh)"])/1000
sourcewisedata.head(10)



In [ ]:
# Group the data by year and month and calculate the sum for each month
monthly_data = sourcewisedata.copy()
monthly_data['YearMonth'] = monthly_data['date'].dt.to_period('M')  # Add a column for year-month

# Sum the data for each month
monthly_summed_data = monthly_data.groupby('YearMonth').sum(numeric_only=True).reset_index()

# Convert the 'YearMonth' column back to datetime if needed
monthly_summed_data['YearMonth'] = monthly_summed_data['YearMonth'].dt.to_timestamp()

print(monthly_summed_data.head())



In [ ]:
# Create a 'YearMonth' column for grouping by month
sourcewisedata['YearMonth'] = sourcewisedata['date'].dt.to_period('M')  # Add a column for year-month

# Group by 'YearMonth' and 'source', and calculate the sum for each group
monthly_summed_data_by_source = sourcewisedata.groupby(['YearMonth', 'source']).sum(numeric_only=True).reset_index()

# Convert the 'YearMonth' column back to datetime if needed
monthly_summed_data_by_source['YearMonth'] = monthly_summed_data_by_source['YearMonth'].dt.to_timestamp()

print(monthly_summed_data_by_source)


In [ ]:
# Save the table to a CSV file
output_file_path = "monthly_summed_data.csv"  # Specify the file name or path
monthly_summed_data.to_csv(output_file_path, index=False)

print(f"Data saved to {output_file_path}")


In [ ]:
#accross different years the lineplot for the electricity generation and corresponding carbon emission can be given with the help of following plot
fig, axes = plt.subplots(1,2, figsize=(13, 7))
sns.lineplot(data=sourcewisedata,x="Year",y="All India",hue="source",ax=axes[0])
sns.lineplot(data=sourcewisedata,x="Year",y="All_India_CO2 (Million Tonnes)" ,hue= "source", ax=axes[1])
axes[0].set_title("Power Generation Across India")
axes[1].set_title("Carbon Emission due to different Energy sources while Power Generation")
plt.show()

In [ ]:
#we can also combine all the renewable and non renewable energy sources to see the amount of carbon emission for both the types of energy sources
energy_map={'Coal & Lignite':"Non Renewable", 'Gas':"Non Renewable", 'Hydro':'Renewable', 'Nuclear':"Non Renewable", 'RES':"Renewable", 'Solar':"Renewable",
       'Wind':"Renewable"}
two_source_data=sourcewisedata
two_source_data["source"]=two_source_data["source"].replace(energy_map)
two_source_data.head(10)

In [ ]:
#let us check the carbon emission for the two energy sources:
sns.lineplot(data=two_source_data,x="Year",y="All_India_CO2 (Million Tonnes)" ,hue= "source")
plt.title("Carbon Emission due to Power generation through Renewable and Non Renewable energy resources")
plt.show()

In [ ]:
#let us see which regions have highest carbon emissions and having renewable or non renewable energy sources in the year 2023
two_source_data_2023=two_source_data[two_source_data['Year']==2023]
two_source_data_2023_grouped=two_source_data_2023.groupby(['source']).sum(numeric_only=True)[['NR','WR','SR','ER','NER','NR_CO2 (Million Tonnes)','WR_CO2 (Million Tonnes)','SR_CO2 (Million Tonnes)','ER_CO2 (Million Tonnes)','NER_CO2 (Million Tonnes)','All_India_CO2 (Million Tonnes)']].reset_index()
two_source_data_2023_grouped

In [ ]:
data=pd.read_csv("Working_table.csv")
data

In [ ]:
# Drop columns with 'Unnamed' in their name
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order, ARDL
from statsmodels.tsa.stattools import adfuller

# Step 1: Transform Variables
data['ln_CO2'] = np.log(data['CO2 (Kilo Tonnes)'])
data['ln_EINT'] = np.log(data['Energy Intensity (MU/Bill_USD)'])
data['ln_ERE'] = np.log(data['Energy_from_RE (MU)'])
data['ln_GDP'] = np.log(data['Annual_GDP(Bill_USD)'])
data['ln_GDP2'] = data['ln_GDP'] ** 2

# Define dependent and independent variables
dependent = data['ln_CO2']
independent_vars = data[['ln_EINT', 'ln_ERE', 'ln_GDP2']]

# Step 2: Check Stationarity (ADF Test)
adf_results = {col: adfuller(data[col].dropna(), autolag='AIC') for col in ['ln_CO2', 'ln_EINT', 'ln_ERE', 'ln_GDP2']}
for col, result in adf_results.items():
    print(f"ADF Test for {col}: p-value = {result[1]}")

# Step 3: ARDL Lag Order Selection
lag_selection = ardl_select_order(
    endog=dependent,
    exog=independent_vars,
    maxlag=1,  # Adjust max lag as needed
    maxorder=1,
    ic='aic'   # Information criterion for lag selection
)

# Inspect lag order
print("Selected ARDL Lag Order:")
if hasattr(lag_selection, 'ar_lags') and hasattr(lag_selection, 'model_orders'):
    print("Dependent Variable Lags (AR Lags):", lag_selection.ar_lags)
    print("Independent Variable Lags:", lag_selection.model_orders)
elif hasattr(lag_selection, 'model_orders'):
    print("Model Orders:", lag_selection.model_orders)
else:
    print("Could not retrieve lag order. Check the lag_selection object structure.")

# Step 4: Fit the ARDL Model
ardl_model = ARDL(
    dependent,
    lags=lag_selection.ar_lags,
    exog=independent_vars,
    trend='c'  # Include a constant term
).fit()

# Display the summary of the ARDL model
print(ardl_model.summary())


In [ ]:
from statsmodels.tsa.ardl import UECM
# Step 6: Cointegration Bounds Test using UECM
uecm_model = UECM(dependent, lags=ardl_model.model.k_ar, exog=independent_vars)
bounds_test = uecm_model.bounds_test()  # Perform bounds test

# Display the bounds test results
print("Cointegration Bounds Test Results:")
print(bounds_test)

# Step 7: Interpretation
if bounds_test.reject_null:
    print("The null hypothesis of no cointegration is rejected. There is cointegration.")
else:
    print("The null hypothesis of no cointegration cannot be rejected. No cointegration.")